# Wrangle OpenStreetMap Data - Tri-Cities, Washington

## Introduction

> This purpose of this project is to take a certain area of land and wrangle the data. Extract and clean the data to make it much more uniform and readable. The area I have chosen is an area called 'Tri-Cities' in Washington State. I chose this area as I have lived here for a long time. The original OpenStreetMap data I am using for this project can be found and exported [Here](https://www.openstreetmap.org/export#map=11/46.2421/-119.1886). I used an API called Overpass API to export this data. The documentation for Overpass API can be found [Here](https://wiki.openstreetmap.org/wiki/Overpass_API). The finished project and all files will be uploaded to [This GitHub Repository](https://github.com/TrikerDev/Wrangle-OpenStreetMap-Data---Tri-Cities---Washington).

# Data Gathering

## Importing

In [2]:
# Importing OSM File and packages

import xml.etree.ElementTree as ET
import pprint
from collections import defaultdict
import re
import csv
import codecs
import sqlite3
import cerberus
import schema
import os
import mysql.connector as mysql
import json


OSM_FILE = "map.osm"

## Gathering Tags

In [63]:
# Getting tags
def get_element(filename, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(filename, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

## Fetching Tags

In [64]:
# Counting and returning tag counts per type of tag
def count_tags(filename):
    tree=ET.iterparse(filename)
    tags={}
    for event,elem in tree:
        if elem.tag not in tags.keys():
            tags[elem.tag]=1
        else:
            tags[elem.tag] = tags[elem.tag]+1
    return tags    
    
with open(OSM_FILE,'rb') as f:
    tags=count_tags(OSM_FILE)
    pprint.pprint(tags)
f.close()

{'bounds': 1,
 'member': 15770,
 'meta': 1,
 'nd': 434301,
 'node': 368263,
 'note': 1,
 'osm': 1,
 'relation': 1431,
 'tag': 158810,
 'way': 38417}


## Finding the 'K' value of each tag and adding to a dictionary

>*  "lower", for tags that contain only lowercase letters and are valid
>*  "lower_colon", for otherwise valid tags with a colon in their names
>*  "problemchars", for tags with problematic characters
>*  "other", for other tags that do not fall into the other three categories

In [65]:
# Adding variables for the tags to be stored under, definitions above
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
    
# Function starting on our dataset
def process_keys_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

# Iterating through tags and adding them up
def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] = keys['problemchars'] + 1
        else:    
            keys['other'] += 1  
    return keys


# Opening file and starting process_keys_map function
with open(OSM_FILE,'rb') as f:
    keys = process_keys_map(OSM_FILE)
    pprint.pprint(keys)

# Closing file
f.close()

# Below is the count of each type of variable

{'lower': 111439, 'lower_colon': 43264, 'other': 4106, 'problemchars': 1}


## Getting number of unique users that contributed to the map data

In [66]:
# Function to count users
def count_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.get('user'):
            users.add(get_user(element))
        element.clear()    
    return users

# Fetching users function
def get_user(element):
    return element.get('user')

# Opening file and starting count_users function
with open(OSM_FILE,'rb') as f:
    users = count_users(OSM_FILE)

print (len(users))

f.close()

# user count below

453


In [67]:
# Printing all unique usernames
pprint.pprint(users)

{'25or6to4',
 '503Greg',
 'AE35',
 'Aaron Lidman',
 'AaronAsAChimp',
 'Acefirst',
 'Adam Brault',
 'Adam Schneider',
 'Adamant1',
 'Akgeo',
 'Aleksandar Matejevic',
 'Alex-a',
 'AmandaCora',
 'Amnesiac9',
 'Amoebabadass',
 'AndjelaS',
 'Andre Engels',
 'Andre68',
 'AndrewSnow',
 'AntonioPigafetta',
 'ArizonaMapper',
 'ArminGh',
 'Audiophase',
 'AzrielB',
 'Azuka',
 'BCNorwich',
 'BadRegEx',
 'Baloo Uriza',
 'BaseballNut51',
 'Belobog',
 'Benny Goodman',
 'Black Cardinal',
 'Brad Meteor',
 'Brian Bradford',
 'Brian Reavis',
 'Brian@Brea',
 'Bryce C Nesbitt',
 'CarniLvr79',
 'Carnildo',
 'Chenshi',
 'Chetan_Gowda',
 'ChrisZontine',
 'Christopher-0118',
 'Chroma187',
 'Claudius Henrichs',
 'DJ Cane',
 'DKDestroyer',
 'Daniel C Berman',
 'DannyAiquipa',
 'DanteVento',
 'DareDJ',
 'Darrell',
 'DaveHansenTiger',
 'Die immer lacht',
 'Dieter Schmeer',
 'Dilys',
 'Dima Ser',
 'Dion Dock',
 'Dr Kludge',
 'E-Layne',
 'ELadner',
 'Ed DeVoe',
 'EdSS',
 'Edward',
 'EmlynSquare',
 'EvanWeiner',
 'Ex

## Getting Street Names

In [68]:
def street_names(filename):
        key='addr:street'
        values=[]
        EL=get_element(filename, tags=('node', 'way', 'relation'))
        for element in EL:
            for tag in element.iter('tag'):
                if tag.attrib['k']==key:
                    values.append(tag.attrib['v'])
            element.clear()
        print (key)
        pprint.pprint(values)
street_names(OSM_FILE)

addr:street
['West 27th Avenue',
 'Hunt Avenue',
 'Fowler Street',
 'Clearwater Avenue',
 'Tapteal Drive',
 'Willamette Ave',
 'Queensgate Drive',
 'Lee Boulevard',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Lee Boulevard',
 'Williams Boulevard',
 'Williams Boulevard',
 'Queensgate Drive',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Lindberg Loop',
 'Serivce Road',
 'Bronco Lane',
 'Bronco Lane',
 'Bronco Lane',
 'Butler Loop',
 'Butler Loop',
 'Airport Way',
 'Airport Way',
 'Aviator Drive',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Terminal Drive',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Airport Way',
 'Airport Way',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Lindberg Loop',
 'Terminal Drive',
 'Terminal Drive',
 'Terminal Drive',
 'Terminal Drive',
 'Terminal Drive',
 'Ter

> The main issue here is that the street name endings are not uniform. For example, some streets end in 'Drive' while others end in 'Dr', or 'Avenue' and 'Ave'. We want all the street name endings to be uniform.

# Data Cleaning

## Auditing Street Names

> Setting values we expect and mappings to them from common endings

In [69]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# The values we expect to see. This is the end result of all street name endings we want
expected = ["Street", "Avenue","Loop", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Freeway","Circle","Strand","Sterling","Way","Highway",
            "Terrace","South","East","West","North","Landing"]

# Mapping shortened version of common street name endings to the ending we want
mapping = {
            " St ": " Street ",
            " St": " Street ",
            " St.": " Street ",
            " ST": " Street ",
            " Rd.": " Road ",
            " Rd ": " Road ",
            " Rd": " Road ",
            " Ave ": " Avenue ", 
            " Ave": " Avenue ", 
            " Ave.": " Avenue ",
            " Av ": " Avenue ", 
            " Dr ": " Drive ",
            " Dr.": " Drive",
            " Dr": " Drive",
            " Pl ": " Place",
            " Pl": " Place",
            " Blvd ": " Boulevard ",
            " Blvd": " Boulevard",
            " Blvd.": " Boulevard",
            " Ct ": " Court ",
            " Ct": " Court ",
            " Ctr": " Center",
            " Pl ": " Place ",
            " Ln ": " Lane ",
            " Cir ": " Circle ",
            " Wy": " Way ",
            " S ": " South ",
            " E ": " East ",
            " W ": " West ",
            " N ": "North"
}

> Adding expected street names to groups and adding unexpected ones to be handled later

In [70]:
# Handling expected and unexpected street names
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

> Sending street names to be audited

In [71]:
# Getting street names and sending them to audit_street_type function
def audit(file):
    file = open(file, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    file.close()
    return street_types

> Updating the ending names of streets to our preferred type

In [72]:
# Function for replace 'old' names with 'new' names
def update_name(name, mapping):
    for key,value in mapping.items():
        if key in name:
            return name.replace(key,value)
    return name 

> Starting the street name auditing process

In [73]:
# Variable for new street names to be stored
st_types = audit(OSM_FILE)

# Printing 
# pprint.pprint(dict(st_types))

#pprint.pprint(dict(st_types))
for st_type, ways in st_types.items():
    for name in ways:
        better_name = update_name(name, mapping)
        print (name, "=>", better_name)

Willamette Ave => Willamette Avenue 
West Kennewick Ave => West Kennewick Avenue 
So. Kent St. => So. Kent Street .
Travis Ct => Travis Court 
E. SR 397 => E. SR 397
North Road 44 => North Road 44
South Washington St => South Washington Street 
North Road 92 => North Road 92
North Road 68 => North Road 68
N Irving Pl => N Irving Place
Indian Ridge Dr => Indian Ridge Drive
Tamarisk Dr => Tamarisk Drive
Road 36 => Road 36
Cottonwood Creek Blvd => Cottonwood Creek Boulevard
S OCTAVE ST => S OCTAVE Street 
Road 72 => Road 72


> Now all the common street endings have been updated to the full endings we want. However, there are some areas that may seem like a problem here. There are several endings with a number, such as 'Road 36'. This is actually okay and a correct name. There are many roads around this area that are numbered such as these. We will ignore changing the numbers of these roads because they are already correct.

# Changing to CSV and Importing to a Database

## Preparing for Database

In [74]:
# Defining schema for SQL Database
schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

## Defining CSV files and fields

In [75]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


SCHEMA = schema


NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

## Shaping elements

In [76]:
# Cleaning and shaping elemnents to Python dict
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  
    
    # if elements tag is a 'node'
    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
      
    # if element tag is a 'way'    
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
                
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

## Helper Functions

In [77]:
# Get Element
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [78]:
# Validate Element
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

In [89]:
# Dict Writer
import sys
if sys.version_info[0] >= 3:
    unicode = str
    
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

## Main Function

In [90]:
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [91]:
# Starting the process
if __name__ == '__main__':
    process_map(OSM_FILE, validate=True)


## Creating Database

# Exploring Data

## Size of CSV Files

In [213]:
#links to the csv files
nodes_csv = 'nodes.csv'
ways_csv = 'ways.csv'
nodestags_csv = 'nodes_tags.csv'
waystags_csv = 'ways_tags.csv'
waysnodes_csv = 'ways_nodes.csv'

# Get the size (in bytes) of specified path  
size_nodes = os.path.getsize(nodes_csv) 
size_ways = os.path.getsize(ways_csv) 
size_nodestags = os.path.getsize(nodestags_csv) 
size_waystags = os.path.getsize(waystags_csv) 
size_waysnodes = os.path.getsize(waysnodes_csv) 
  
# Print the size (in bytes) of specified path  
print("Size (In bytes) of '%s':" %nodes_csv, size_nodes)
print("Size (In bytes) of '%s':" %ways_csv, size_ways)
print("Size (In bytes) of '%s':" %nodestags_csv, size_nodestags)
print("Size (In bytes) of '%s':" %waystags_csv, size_waystags)
print("Size (In bytes) of '%s':" %waysnodes_csv, size_waysnodes)

Size (In bytes) of 'nodes.csv': 40543512
Size (In bytes) of 'ways.csv': 3088426
Size (In bytes) of 'nodes_tags.csv': 1526190
Size (In bytes) of 'ways_tags.csv': 5887614
Size (In bytes) of 'ways_nodes.csv': 13413030


## Number of Unique Users